#### Core 1 — Problem Definition

이 프로젝트는 보험 의사결정의 실패 원인을
‘예측 성능 부족’이 아니라
**상태(state)를 다루지 못한 구조적 한계**로 정의한다.

보험은 사건(event)을 맞히는 문제가 아니라,
개인의 **상태 변화(state transition)를 관리하는 문제**다.

본 Core에서는 모델링이나 예측을 수행하지 않는다.
오직 상태가 데이터로서 관측 가능함을 확인하고,
이후 모든 Core의 출발점을 고정한다.

In [2]:
import pandas as pd
import numpy as np

# 원본 상태 시계열 데이터 로드
df = pd.read_csv("../data/raw/health_timeseries_core_state.csv")

print("Data shape:", df.shape)
df.head()
df.info()

Data shape: (943, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             943 non-null    int64  
 1   date                943 non-null    object 
 2   mean_hr             8 non-null      float64
 3   hr_std              8 non-null      float64
 4   steps               943 non-null    int64  
 5   calories            943 non-null    int64  
 6   sleep_minutes       413 non-null    float64
 7   health_state_level  670 non-null    float64
 8   health_state_speed  490 non-null    float64
 9   health_state_index  480 non-null    float64
dtypes: float64(6), int64(3), object(1)
memory usage: 73.8+ KB


In [3]:
# 핵심 상태 컬럼만 확인
state_cols = [
    "user_id",
    "date",
    "health_state_index"
]

df[state_cols].head()

,user_id,date,health_state_index
0,1503960366,2016-04-12,-0.002802
1,1503960366,2016-04-13,0.452790
2,1503960366,2016-04-14,NaN
3,1503960366,2016-04-15,NaN
4,1503960366,2016-04-16,NaN


In [4]:
# 사용자 수 확인
df["user_id"].nunique()

# 상태값 분포
df["health_state_index"].describe()

count    480.000000
mean       0.009907
std        0.860060
min       -5.838454
25%       -0.329480
50%        0.040657
75%        0.372057
max        4.325406
Name: health_state_index, dtype: float64

개별 사용자 상태 변화 예시 확인

In [5]:
sample_user = df["user_id"].iloc[0]

sample_df = (
    df[df["user_id"] == sample_user]
    .sort_values("date")
    .reset_index(drop=True)
)

sample_df[["date", "health_state_index"]].head(10)

,date,health_state_index
0,2016-04-12,-0.002802
1,2016-04-13,0.452790
2,2016-04-14,NaN
3,2016-04-15,NaN
4,2016-04-16,NaN
5,2016-04-17,NaN
6,2016-04-18,NaN
7,2016-04-19,NaN
8,2016-04-20,NaN
9,2016-04-21,NaN


상태 개념 선언

## Core 1 결론

- 개인의 건강 상태는 시간에 따라 변화하는 **연속적인 상태 변수**로 관측된다.
- 이 상태는 단일 사건(event)이 아니라 **시계열적 맥락**을 가진다.
- 따라서 보험 의사결정은 사건 예측 문제가 아니라,
  **상태 변화 관리 문제**로 재정의되어야 한다.

이 선언은 이후 모든 Core에서 변경되지 않는다.